In [1]:
import os
import pyspark
from pyspark.sql import SparkSession

In [2]:
ip_path = '../data'
filename = 'CompleteDataset.csv'

In [3]:
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()


24/05/05 18:41:51 WARN Utils: Your hostname, Shubhams-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.107 instead (on interface en0)
24/05/05 18:41:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 18:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df  = spark.read.format('csv').load(os.path.join(ip_path,filename),inferSchema=True,header=True)

Basic Fuction

In [14]:
df.rdd.getNumPartitions()
df2 = df.repartition(4)
df2.rdd.getNumPartitions()

24/05/05 18:48:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Acceleration, Aggression, Agility, Balance, Ball control, Composure, Crossing, Curve, Dribbling, Finishing, Free kick accuracy, GK diving, GK handling, GK kicking, GK positioning, GK reflexes, Heading accuracy, Interceptions, Jumping, Long passing, Long shots, Marking, Penalties, Positioning, Reactions, Short passing, Shot power, Sliding tackle, Sprint speed, Stamina, Standing tackle, Strength, Vision, Volleys, CAM, CB, CDM, CF, CM, ID, LAM, LB, LCB, LCM, LDM, LF, LM, LS, LW, LWB, Preferred Positions, RAM, RB, RCB, RCM, RDM, RF, RM, RS, RW, RWB, ST
 Schema: _c0, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Acceleration, Aggression, Agility, Balance, Ball control, Composure, Crossing, Curve, Dribbling, Finishing, Free kick accuracy, GK diving, GK ha

4

In [15]:
df2.columns
df2.withColumnRenamed('_c0','Id')


DataFrame[Id: int, Name: string, Age: int, Photo: string, Nationality: string, Flag: string, Overall: int, Potential: int, Club: string, Club Logo: string, Value: string, Wage: string, Special: int, Acceleration: string, Aggression: string, Agility: string, Balance: string, Ball control: string, Composure: string, Crossing: string, Curve: string, Dribbling: string, Finishing: string, Free kick accuracy: string, GK diving: string, GK handling: string, GK kicking: string, GK positioning: string, GK reflexes: string, Heading accuracy: string, Interceptions: string, Jumping: string, Long passing: string, Long shots: string, Marking: string, Penalties: string, Positioning: string, Reactions: string, Short passing: string, Shot power: string, Sliding tackle: string, Sprint speed: string, Stamina: string, Standing tackle: string, Strength: string, Vision: string, Volleys: string, CAM: double, CB: double, CDM: double, CF: double, CM: double, ID: int, LAM: double, LB: double, LCB: double, LCM: 

In [25]:
df2.select('Id','Name').where((df2['Id']>191104) & (df2['Id']<215061)).sort('Name','Id').show()

+------+-------------+
|    Id|         Name|
+------+-------------+
|195540|     A. Abbas|
|208915|     A. Abbas|
|198076| A. Abdennour|
|199282|   A. Abrashi|
|198740|    A. Acosta|
|201223|    A. Acquah|
|204845|    A. Affane|
|197678|   A. Ahamada|
|193029| A. Ajdarević|
|203458|     A. Ajeti|
|205234|   A. Akaichi|
|201214|  A. Akhyadov|
|191847|   A. Al Asta|
|210924|A. Al Barakah|
|191262|A. Al Dawsari|
|210902|A. Al Dawsari|
|208926|  A. Al Enezi|
|210898|  A. Al Fahad|
|210183|  A. Al Fahmi|
|209223|  A. Al Faraj|
+------+-------------+
only showing top 20 rows



Advanded Funntion

In [26]:
df2.createOrReplaceTempView('fooltball_df')

24/05/05 19:17:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [29]:
sql_query = """ Select Age, count(*) as count
                from  fooltball_df
                where overall>70
                group by age 
                order by age"""

result = spark.sql(sql_query)
result.show()

+---+-----+
|Age|count|
+---+-----+
| 17|    1|
| 18|   13|
| 19|   41|
| 20|  105|
| 21|  160|
| 22|  246|
| 23|  296|
| 24|  369|
| 25|  486|
| 26|  418|
| 27|  444|
| 28|  395|
| 29|  474|
| 30|  359|
| 31|  272|
| 32|  206|
| 33|  283|
| 34|  101|
| 35|   74|
| 36|   57|
+---+-----+
only showing top 20 rows



In [30]:
def convert_uppercase(record):
    if len(record)>10:
        return record.upper()
    else:
        return record


spark.udf.register('sql_upper',convert_uppercase)

sql_query = """ Select sql_upper(Name) as new_name, sql_upper(Nationality) as new_nationality
                from fooltball_df"""

result = spark.sql(sql_query)
result.show()

+----------------+---------------+
|        new_name|new_nationality|
+----------------+---------------+
|DANI ROCHELINHAS|         Brazil|
|         D. Léon|         France|
|      A. Losilla|         France|
|    KIM JONG WOO| KOREA REPUBLIC|
|         A. Yoda|         France|
|        H. Sacko|         France|
|     L. CARTELDO|          Chile|
|       J. Bacuna|    NETHERLANDS|
|    S. GALLAGHER|        England|
|      A. Baredia|          Chile|
|        S. Clark|  UNITED STATES|
|       M. Araujo|           Peru|
|        S. Byram|        England|
|       A. Fidler|         Russia|
|      H. Çinemre|         Turkey|
|  C. OIKONOMIDIS|      Australia|
|      S. Ulreich|        Germany|
|     JOO MIN KYU| KOREA REPUBLIC|
|  THIAGO CARDOSO|         Brazil|
|       Guilherme|         Brazil|
+----------------+---------------+
only showing top 20 rows

